In [1]:
import sys
from pathlib import Path
import csv 
import pandas as pd
import re

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))

from utilidades.constantes import  ARCHIVO_UNIFICADO_PATH, ARCHIVO_MOSTRADOR_FINAL_PATH, ARCHIVO_FACTURACION_FINAL_PATH

UNIFICAR MOSTRADOR CON FACTURAS (LIMPIAR ARCHIVOS PRIMERO)

In [2]:
# Dataset de resto de las ventas
df_resto = pd.read_csv(ARCHIVO_FACTURACION_FINAL_PATH)

# Dataset del mostrador
df_mostrador = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH)

# Seleccionamos columnas y renombramos
df_resto_simple = df_resto[['fecha_emision', 'denominacion', 'TotalVenta_Ajustado','cuit_receptor','Anio','Mes','importe_total']].rename(columns={
    'fecha_emision': 'fecha',
    'denominacion': 'cliente',
    'TotalVenta_Ajustado': 'monto ajustado',
    'cuit_receptor': 'cuit',
    'Anio': 'anio',
    'Mes': 'mes',
    'importe_total': 'monto'
})

df_mostrador_simple = df_mostrador[['Fecha', 'Cliente', 'TotalVenta_Ajustado', 'Producto','CUIT','Anio','Mes','TotalVenta']].rename(columns={
    'Fecha': 'fecha',
    'Cliente': 'cliente',
    'TotalVenta_Ajustado': 'monto ajustado',
    'Producto': 'producto',
    'CUIT': 'cuit',
    'Anio': 'anio',
    'Mes': 'mes',
    'TotalVenta': 'monto'
})

# Asignamos origen 'servicio' a facturación
df_resto_simple['origen'] = 'servicio'

# Listas de productos (convertidas a minúsculas)
decoracion_items = ['ALMOHADON','APOYA CUCHARA','AZUCARERA','BANCO TRES PATAS', 'BANCO RECTANGULAR', 'COLGANTES DE CERAMICA','COLGANTES VARIOS','CORAZON CHICO','CORAZON GRANDE','CORAZON MEDIANO','CUENCO CHICO','CUENCO MEDIANO','CUENCO MINI','CUENCO GRANDE','JARRA','JARRITO/ MATE/ VASITO','JARRO','LECHERITA','MACETA CHICA','MACETA MEDIANA','PINGUINO LITRO','PINGUINO MEDIO LITRO','PLATO 24CM','PLATO 30CM','PLATO POSTRE','PORTA CUCHARA GRANDE','PORTA VELA','POSE TORTA','REGALERÍA','REGALERIA VARIOS','TAZA SOLA','TAZA SOLA ARTESANAL','TAZON ARTESANAL','TAZON PARA LANA','TELAS','TETERA','VELA', 'velas','TASA CON PLATO MOLDE','TAZA CON PLATO ARTESANAL']
decoracion_items = [item.lower() for item in decoracion_items]

restobar_items = ['RESTOBAR','restovar']
restobar_items = [item.lower() for item in restobar_items]

# Función para asignar origen
def clasificar_origen(producto):
    producto_lower = str(producto).lower()
    if any(item in producto_lower for item in decoracion_items):
        return 'decoracion'
    elif any(item in producto_lower for item in restobar_items):
        return 'restobar'
    else:
        return 'mostrador'

# Asignar origen a mostrador
df_mostrador_simple['origen'] = df_mostrador_simple['producto'].apply(clasificar_origen)

"""# Eliminamos la columna producto si no la querés en el final
df_mostrador_simple = df_mostrador_simple.drop(columns=['producto'])"""

# Concatenamos
df_unido = pd.concat([df_resto_simple, df_mostrador_simple], ignore_index=True)

# Ajustamos fecha
df_unido['fecha'] = df_unido['fecha'].astype(str).str.split().str[0]

# Exportamos
df_unido.to_csv(ARCHIVO_UNIFICADO_PATH, index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_45484\176857561.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mostrador = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH)
